<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js006_ArMD3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [4]:
#@title js006_ArMD3D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023 Mitsuru Ikeuchi
#  home page: https://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.08.18 created,  last updated on 2023.12.19
#


# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<title>js006_ArMD3D</title>
<meta charset="utf-8">
<script type="text/javascript">

'use strict';

/* --------------------
//
//  js006_ArMD3D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.05.03 created, last updated on 2018.11.11
//    ver 0.0.1  2019.01.10 v1, last updated on 2021.04.26
//    ver 0.0.2  2021.10.28 v2, last updated on 2021.10.28
//    ver 0.0.3  2023.02.14 v3, last updated on 2023.08.11
//
// -------------------- molecular dynamics 3D - Lennard-Jones potential
//
//    method: velocity Verlet Algorithm
//      (1) vi = vi + (Fi/mi)*(0.5dt)
//      (2) ri = ri + vi*dt
//      (3) calculation Fi <- {r1,r2,...,rn} Fi=sum(Fij,j=1 to n),Fij=F(ri-rj)
//      (4) vi = vi + (Fi/mi)*(0.5dt)
//      goto (1)
//
//    potential: Lennard-Jones V(r) = 4*epsilon*((sigma/r)^12-(sigma/r)^6)
//           force F(r) = -dV(r)/dr = 24.0*epsilon*r6*(2.0*r6-1.0)/r, r6=(sigma/r)^6
//
// --------------------
*/

const ArMD3D = (function(){ // ====================  ArMD3D Module  ====================

	const g_AMU = 1.66053904e-27;		// (kg) atomic mass unit
	const g_kB = 1.380649e-23;			// (J/K) Boltzmann's constant
	const g_nMax = 500;					// array max

	let g_nParticles = 200;				// number of particles
	let g_sysTime = 0.0;				// (s) system time
	let g_timeStep = 5.0*1.0e-15;		// (s) time step dt
	let g_xMax = 6.0E-9;				// (m) x-Box size
	let g_yMax = 6.0E-9;				// (m) y-Box size
	let g_zMax = 6.0E-9;				// (m) z-Box size
	let g_kineticEnergy = 0.0;			// (J) total kinetic energy
	let g_potentialEnergy = 0.0;		// (J) total potential energy

	const g_xx = dim1( g_nMax );		// (m) x-component of i-th particle position
	const g_yy = dim1( g_nMax );		// (m) y-component of i-th particle position
	const g_zz = dim1( g_nMax );		// (m) z-component of i-th particle position
	const g_vx = dim1( g_nMax );		// (m/s) x-component of i-th particle velocity
	const g_vy = dim1( g_nMax );		// (m/s) y-component of i-th particle velocity
	const g_vz = dim1( g_nMax );		// (m/s) z-component of i-th particle velocity
	const g_ffx = dim1( g_nMax );		// (N) x-component of total force applied i-th particle
	const g_ffy = dim1( g_nMax );		// (N) y-component of total force applied i-th particle
	const g_ffz = dim1( g_nMax );		// (N) z-component of total force applied i-th particle

	function dim1( n ) {
		return new Float64Array( n );
	}

	//--- material data

	const g_mass = 39.948*g_AMU;		// (kg) mass of Ar
	const g_sigma = 3.418e-10;			// (m) Lennard-Jones potential sigma for Ar
	const g_epsilon = 124.0*g_kB;		// (J) Lennard-Jones potential epsilon FOR Ar


	// --------------------  set initial condition  --------------------

	function setInitialCondition( nn, BoxSizeInNM, contTemp ) {
		g_sysTime = 0.0;
		g_xMax = BoxSizeInNM*1.0e-9;
		g_yMax = BoxSizeInNM*1.0e-9;
		g_zMax = BoxSizeInNM*1.0e-9;
		g_nParticles = setParticles(nn);
		ajustVelocity(contTemp);
	}

	function setParticles( nn ) {
		const sigma = g_sigma;
		let nParticles = nn;

		for ( let j=0; j<nn; j++ ) {
			let loopCount = 0;
			let i;
			do {
				g_xx[j] = (g_xMax-1.2*sigma)*Math.random() + 0.6*sigma;
				g_yy[j] = (g_yMax-1.2*sigma)*Math.random() + 0.6*sigma;
				g_zz[j] = (g_zMax-1.2*sigma)*Math.random() + 0.6*sigma;
				for ( i=0; i<j; i++ ) {
					if ((g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j])
						+(g_zz[i]-g_zz[j])*(g_zz[i]-g_zz[j]) < 1.4*sigma*sigma) break;
				}
				loopCount = loopCount + 1
				if (loopCount>1000) break;
			} while (i<j);
			if (loopCount>1000) {
				nParticles = j - 1;
				break;
			}
		}
		for ( let i=0; i<nParticles; i++ ) {
			g_vx[i] = 200.0*normalRandom3();
			g_vy[i] = 200.0*normalRandom3();
			g_vz[i] = 200.0*normalRandom3();
			g_ffx[i] = 0.0;
			g_ffy[i] = 0.0;
			g_ffz[i] = 0.0;
		}
		return nParticles;
	}

	// normal distributed random number: -3.0 <= normalRandom3() < 3.0
	function normalRandom3() {
		return (Math.random()+Math.random()+Math.random()+Math.random()+Math.random()+Math.random()-3.0);
	}


	// --------------------  time evolution  --------------------

	function timeEvolution(tempMode,contTemp) {

		if (tempMode==1) ajustVelocity(contTemp);
		for (let i=0; i<20; i++) {
			g_sysTime += g_timeStep;
			moveParticles(g_timeStep);
		}
	}

	function moveParticles(dt) {
		const nn = g_nParticles, a = 0.5*dt/g_mass;

		for (let i=0; i<nn; i++) {
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_vz[i] += a*g_ffz[i];
			g_xx[i] += g_vx[i]*dt;
			g_yy[i] += g_vy[i]*dt;
			g_zz[i] += g_vz[i]*dt;
		}
		calcForce();
		for (let i=0; i<nn; i++) {
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_vz[i] += a*g_ffz[i];
		}
		g_kineticEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_kineticEnergy += 0.5*g_mass*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]+g_vz[i]*g_vz[i]);
		}
	}

	function calcForce() {
		const nn = g_nParticles, s05 = 0.5*g_sigma;

		g_potentialEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_ffx[i]=0; g_ffy[i]=0; g_ffz[i]=0;
		}

		for (let i=0; i<nn-1; i++) {
			for (let j=i+1; j<nn; j++) {
				const xij=g_xx[i]-g_xx[j], yij=g_yy[i]-g_yy[j], zij=g_zz[i]-g_zz[j];
				const r = Math.sqrt(xij*xij+yij*yij+zij*zij);
				// calc. force and potential energy
				const ri = (g_sigma/r);
				const r6 =ri*ri*ri*ri*ri*ri;
				g_potentialEnergy += 4.0*g_epsilon*r6*(r6-1.0); // V(r) = 4.0*epsilon*((sigma/r)^12-(sigma/r)^6)
				const f = 24.0*g_epsilon*r6*(2.0*r6-1.0)/r; // F(r) = 24.0*epsilon*r6*(2.0*r6-1.0)/r, r6=(sigma/r)^6
				//
				const fxij = f*xij/r, fyij = f*yij/r, fzij = f*zij/r;
				g_ffx[i] += fxij;
				g_ffy[i] += fyij;
				g_ffz[i] += fzij;
				g_ffx[j] -= fxij;
				g_ffy[j] -= fyij;
				g_ffz[j] -= fzij;
			}
		}
		for (let i=0; i<nn; i++) {
			g_ffx[i] += boundaryForce(g_xx[i]+s05)+boundaryForce(g_xx[i]-g_xMax-s05);
			g_ffy[i] += boundaryForce(g_yy[i]+s05)+boundaryForce(g_yy[i]-g_yMax-s05);
			g_ffz[i] += boundaryForce(g_zz[i]+s05)+boundaryForce(g_zz[i]-g_zMax-s05);
		}
	}

	// boundary:L-J type; epsilon = 0.5*epsilonOfAr, sigma = sigmaOfAr
	function boundaryForce(r) {
		const ri = (g_sigma/r);
		const r6 = ri*ri*ri*ri*ri*ri;
		g_potentialEnergy += 4.0*0.5*g_epsilon*r6*(r6-1.0);
		return (24.0*0.5*g_epsilon*r6*(2.0*r6-1.0)/r);
	}


	// --------------------  utility  --------------------

	function systemTemperature() {
		const nn = g_nParticles;

		let ek=0.0; // kinetic energy (J)
		for (let i=0; i<nn; i++) {
			ek = ek + 0.5*g_mass*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]+g_vz[i]*g_vz[i]);
		}
		return 2.0*ek/(3.0*nn*g_kB); //2D: E/N=kT, 3D: E/N=(3/2)kT
	}

	function ajustVelocity(temp) {
		const nn=g_nParticles;
		const a = Math.sqrt(temp/systemTemperature());
		for (let i=0; i<nn; i++) {
			g_vx[i] = a*g_vx[i];
			g_vy[i] = a*g_vy[i];
			g_vz[i] = a*g_vz[i];
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( nn, BoxSizeInNM, contTemp )
		evolve:			timeEvolution,			// timeEvolution( tempMode, contTemp)

		setTemp:		ajustVelocity,			// ajustVelocity( contTemp )

		getSysParam:	function() { return [ g_nParticles, g_timeStep, g_xMax, g_yMax, g_zMax ]; },
		getNow:			function() { return [ g_sysTime, systemTemperature(), g_kineticEnergy, g_potentialEnergy ]; },
		getMolecData:	function() { return [ g_mass, g_sigma, g_epsilon, "Ar", 0xaadd00 ]; },
		getPosition:	function(i) { return [ g_xx[i], g_yy[i], g_zz[i] ]; },
		getVelocity:	function(i) { return [ g_vx[i], g_vy[i], g_vz[i] ]; },
		getForce:		function(i) { return [ g_ffx[i], g_ffy[i], g_ffz[i] ]; },
	};

})(); // ====================  ArMD3D Module end  ====================


const js006 = (function(){ // ====================  js Module  ====================

	const theModule = ArMD3D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;
	let ctx;

	let v_theme = 2; // no use
	let v_nn = 200;
	let v_BoxSizeInNM = 6.0;
	let v_tempMode = 0;			// 0: adiabatic, 1: temp.control
	let v_contTemp = 300.0;

	let p_nParticles, p_timeStep, p_xMax, p_yMax, p_zMax; // = theModule.getSysParam();
	let sysTime, sysTemp, kineticEnergy, potentialEnergy;
  let kindList = [];
  let xxList = [];
	let yyList = [];
  let zzList = [];
	let vxList = [];
	let vyList = [];
  let vzList = [];

	let dispMode = 0;
	let resetFlag = true;
	let pauseFlag = false;

	let breakFlag = false;
	let perticleFlag = true;
	let timeStamp = 0.0;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics
		}
	}


	function animate() {
		if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_nn, v_BoxSizeInNM, v_contTemp );
			[ p_nParticles, p_timeStep, p_xMax, p_yMax, p_zMax ] = theModule.getSysParam();
			// g3d.init( nParticles, xMax, yMax, zMax, xCanvasSize, yCanvasSize, xBoxSize, yShift )
			g3d.init( p_nParticles, p_xMax, p_yMax, p_zMax, xCanvasSize, yCanvasSize, 300, 20 );
			perticleFlag = true;
		}

		if ( !pauseFlag ) theModule.evolve( v_tempMode, v_contTemp );

		draw( ctx );

		if ( perticleFlag ) setParticlesData();

		requestAnimationFrame(animate);
	}

  function setParticlesData() {
		timeStamp = sysTime;
		kindList = [];
		xxList = [];
		yyList = [];
		zzList = [];
		vxList = [];
		vyList = [];
		vzList = [];

    for (let i=0; i<p_nParticles; i++) {
			let knd, x, y, z, vx, vy, vz;
      //knd = theModule.getKind(i);
      kindList[i] = 2; // kind = 2: Ar
			[ x, y, z ] = theModule.getPosition(i);
			xxList[i] = x;
			yyList[i] = y;
      zzList[i] = z;
			[ vx, vy, vz ] = theModule.getVelocity(i);
			vxList[i] = vx;
			vyList[i] = vy;
      vzList[i] = vz;
		}
	}


	// --------------------  draw  --------------------

	function draw( ctx ) {
		[ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = theModule.getNow();
		let mass, sigma, epsilon, str, colour;
		[ mass, sigma, epsilon, str, colour ] = theModule.getMolecData();

		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		draw3D( ctx, p_nParticles, sigma );

		// caption
		const yTextPos = yCanvasSize-45; // in pixel
		ctx.fillStyle = "#888888";
		ctx.fillText(`time =${(sysTime*1.0e12).toFixed(1)} (ps)`, 40, yTextPos);
		ctx.fillText(`Temp =${sysTemp.toFixed(1)} (K)`, 260, yTextPos);
		ctx.fillText(`Ar, N =${p_nParticles}`, 40, yTextPos+20);
		ctx.fillText("tempMode : adiabatic", 260, yTextPos+20);
		ctx.fillText(`Box =${(p_xMax*1.0e9).toFixed(1)}x${(p_yMax*1.0e9).toFixed(1)}x${(p_zMax*1.0e9).toFixed(1)} (nm)`,
						40, yTextPos+40);
		ctx.fillText(`Energy = ${(kineticEnergy+potentialEnergy).toExponential(4)} (J)`, 260, yTextPos+40);
	}

	function draw3D( ctx, nn, sigma ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc(); // g3d sc, xp, yp

		const ArBallColor = [ 0xb4, 0xff, 0x00 ];

		// draw 3D
		g3d.set3DRotatedObjects(0.0); //(rotateRateOfAyInDegree) eg. 0.0 or 0.5
		g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow

		for (let i=0; i<nn; i++) {
			const j = g3d.srtzix[i];
			const zr = (g3d.ppz[j]/g_zMax+1.27)/3.0; // -0.73 < g3d.ppz[j]/g_zMax < 1.73
			const x = g3d.ppx[j]*sc+xp, y = g3d.ppy[j]*sc+yp, r = 0.5*sigma*sc;
			g3d.drawDisc( ctx, x, y, r, ballColor( ArBallColor, zr ) );
			g3d.strokeCircle( ctx, x, y, r, ballColor( ArBallColor, 0.7*zr ) );
		}

		g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); // yellow
		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	}

	function ballColor( ballColorV3, zr ) {
		const r = Math.floor(ballColorV3[0]*zr);
		const g = Math.floor(ballColorV3[1]*zr);
		const b = Math.floor(ballColorV3[2]*zr);
		return `rgb(${r},${g},${b})`;
	}


	// --------------------  3D graphics (particles) module  --------------------
	//
	// ver 0.0.1  2018.12.23  last updated on 2023.04.10
	// ver 0.0.2  2023.05.02  last updated on 2023.06.02

	let g_nParticles, g_xMax, g_yMax, g_zMax, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift; // gloval in g3d

	const g3d = {};				// namespace of graphic 3D module
	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.cx0 = 0.0;				// x-rotate center (3D graphics)
	g3d.cy0 = 0.0;				// y-rotate center (3D graphics)
	g3d.cz0 = 0.0;				// z-rotate center (3D graphics)
	g3d.Ax = -Math.PI/12.0;		// x-rotate angle around x-axis
	g3d.Ay = -Math.PI/12.0;		// y-rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.ppx = [];				// (m) rotated x-component of i-th particle position
	g3d.ppy = [];				// (m) rotated y-component of i-th particle position
	g3d.ppz = [];				// (m) rotated z-component of i-th particle position
	g3d.srtzix = [];			// z(depth)-sorted index: srtzix[1], srtzix[2],...,srtzix[g_nParticles]

	g3d.xApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// x-apex of box
	g3d.yApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// y-apex of box
	g3d.zApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// z-apex of box
	g3d.pxApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// rotated x-apex of box
	g3d.pyApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// rotated y-apex of box
	g3d.pzApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// rotated z-apex of box
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
				   [2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};
	g3d.setMouseOnCanvas.defined = false;

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid (particles)

	// g3d.init( nParticles, xMax, yMax, zMax, xCanvasSize, yCanvasSize, xBoxSize, yShift )
	g3d.init = function( nParticles, xMax, yMax, zMax, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_nParticles = nParticles;
		g_xMax = xMax;
		g_yMax = yMax;
		g_zMax = zMax;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.cx0 = 0.5*g_xMax;		// x-rotate center (3D graphics)
		g3d.cy0 = 0.5*g_yMax;		// y-rotate center (3D graphics)
		g3d.cz0 = 0.5*g_zMax;		// z-rotate center (3D graphics)
	}

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = xCanvasSize/2, yCenter = yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_xMax)*g_yMax;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = (xBoxSize/g_xMax)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotatedObjects = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate angle and rotate center(=box center)
		g3d.rotateXY();         // g_xx(i),g_yy(i),g_zz(i) rotate--> ppx(i),ppy(i),ppz(i)
		g3d.sortz();            // sort ppz(i) : ppz(srtzix(1)),ppz(srtzix(2)),...,ppz(srtzix(g_nParticles))
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g_xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g_yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g_zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g_xMax;
		g3d.cy0 = 0.5*g_yMax;
		g3d.cz0 = 0.5*g_zMax;
	};

	g3d.rotateXY = function() { //particles and box apex
		const nn = g_nParticles;
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<nn; i++) {
			let xi, yi, zi;
			[ xi, yi, zi ] = theModule.getPosition(i);
			const y = g_yMax-yi-cy0;
			g3d.ppx[i] = cosAy*(xi-cx0)+sinAy*(sinAx*y+cosAx*(zi-cz0)) + cx0;
			g3d.ppy[i] = cosAx*y-sinAx*(zi-cz0) + cy0;
			g3d.ppz[i] =-sinAy*(xi-cx0)+cosAy*(sinAx*y+cosAx*(zi-cz0)) + cz0;
		}
		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.rotateXYPos = function(xi,yi,zi) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;
		const y = g_yMax-yi-cy0;
		const rotx = cosAy*(xi-cx0)+sinAy*(sinAx*y+cosAx*(zi-cz0)) + cx0;
		const roty = cosAx*y-sinAx*(zi-cz0) + cy0;
		//const rotz =-sinAy*(xi-cx0)+cosAy*(sinAx*y+cosAx*(zi-cz0)) + cz0;
		return [ rotx, roty ];
	};

	g3d.sortz = function() {
		const nn=g_nParticles;
		for (let i=0; i<nn; i++) {
			g3d.srtzix[i] = i;
		}
		g3d.qSort(0,nn-1);
	};

	g3d.qSort = function(le,ri) {
		let i,j, pv,w;
		if (ri>le) {
			i = le-1;
			j = ri;
			pv = g3d.ppz[g3d.srtzix[ri]];
			while (1) {
				do {
					i=i+1;
				} while (pv>g3d.ppz[g3d.srtzix[i]]);
				do {
					j=j-1;
				} while (j>i && g3d.ppz[g3d.srtzix[j]]>pv);
				if (j<=i) break;
				w=g3d.srtzix[i]; g3d.srtzix[i]=g3d.srtzix[j]; g3d.srtzix[j]=w;
			}
			w=g3d.srtzix[i]; g3d.srtzix[i]=g3d.srtzix[ri]; g3d.srtzix[ri]=w;
			g3d.qSort(le,i-1);
			g3d.qSort(i+1,ri);
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark == 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark == 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1 = g3d.pxApex[iApex]*sc+xp;
		const y1 = g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2 = g3d.pxApex[iApex]*sc+xp;
		const y2 = g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function( ctx, x, y, r, color ) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	g3d.strokeCircle = function( ctx, x, y, r, color ) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.stroke();
	}

	// --------------------  3D graphics (particles) module end  --------------------


	// -------------------- dom control

	function setTemperature(temp) {
		theModule.setTemp(temp);
	}

	function pause() { // button [pause/go]
		pauseFlag = ( pauseFlag===false );
	}

	function viewHome() {
		g3d.setRotateAngle(-10,-15); // in degree
		g3d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetTemperature(temp) {
    theModule.setTemp(temp);
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, sysTemp, kineticEnergy, potentialEnergy ];
  }

  function pygetParticlesList() {
    perticleFlag = false;
    return [ timeStamp, kindList, xxList, yyList, zzList, vxList, vyList, vzList ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		pause:			pause,			// pause();

		viewHome:		viewHome,		// viewHome()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetTemperature: pysetTemperature, // pysetTemperature( temp )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, sysTemp, kineticEnergy, potentialEnergy ]
		pygetParticlesList: pygetParticlesList, //() :return [ timeStamp, kindList, xxList, yyList, zzList, vxList, vyList, vzList ]
	};

})(); // =============  js006 module end  =========================================================================


const js = js006;
//window.addEventListener('load', js.main );
js.main();


//--------------------  end of javaScript

</script>
<style type="text/css">
body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>
</head>

<body>
<p>[js006] Ar in the Box (Molecular Dynamics 3D)</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="480" HEIGHT="480"></canvas><br>

<button onClick="js.pause()">pause/go</button>
    <span style="margin-right: 160px;"></span>
<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="400" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# exec html-js code, get data and print

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  # get data and display
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print( f' i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  energy = {energy:9.6e} (J)' )
  time.sleep(2)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")


In [ ]:
# system cool down

import time
import matplotlib.pyplot as plt

# exec html-js code
exec_html_js()
print("-- start --")

# prepare data list
timeList = []
tempList = []
energyList = []

# system cool down
for i in range(20):
  # set temperature
  temp = 300 - 15*i
  eval_js( 'js.pysetTemperature({})'.format(temp) )
  time.sleep(3)

  # get data and display
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy

  timeList.append(sysTime*1e12)
  tempList.append(sysTemp)
  energyList.append(energy*1e20)

  if i%2==0:
    print(f'\t time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  energy = {energy:9.6e} (J)' )
  time.sleep(3)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

# plot temp, energy
print("-- plot ---")
plt.plot( timeList, tempList, 'xb-' )
plt.xlabel('time (ps)')
plt.ylabel('temp (K)')
plt.show()

plt.plot( timeList, energyList, 'og-' )
plt.xlabel('time (ps)')
plt.ylabel('energy ( x10^-20 J )')
plt.show()

In [ ]:
# get particles data and save

import time
import numpy as np

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  # get data and display
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print( f' i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  energy = {energy:9.6e} (J)' )
  time.sleep(1)

# get particle data
print("-- get particles data --")
[ timeStamp, kindList, xxList, yyList, zzList, vxList, vyList, vzList ] = eval_js('js.pygetParticlesList()')
print(f'time stamp = {timeStamp*1e12:>8.3f} (ps)')

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

# convert [ kindList, xxList, yyList, vxList, vyList ] to np_data
np_data = np.array([ kindList, xxList, yyList, zzList, vxList, vyList, vzList ])
print("shape of  np_data :", np_data.shape )

# save np_data
print("-- save as 'js006_data.npy'" )
np.save('js006_data.npy', np_data )


In [29]:
# save np_data as csv format (for Excel)

np.savetxt('js006_data.csv', np_data.T, delimiter=',')


In [ ]:
# Ar atom position in the box - matplotlib

import numpy as np
import matplotlib.pyplot as plt

# load np_data <-- np.array([ kindList, xxList, yyList, zzList, vxList, vyList, vzList ])
loaded_data = np.load('js006_data.npy')
X = loaded_data[1]*1e9  # (nm)
Y = loaded_data[2]*1e9  # (nm)
Z = loaded_data[3]*1e9  # (nm)
U = loaded_data[4]      # (m/s)
V = loaded_data[5]      # (m/s)
W = loaded_data[6]      # (m/s)
E = U**2 + V**2 + W**2  # 3 k T ~ m v^2 --> E ~ T

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X, Y, Z, c=E, s=50)
ax.set_xlabel('x (nm)')
ax.set_ylabel('y (nm)')
ax.set_zlabel('z (nm)')
plt.title("Ar in the Box")
plt.show()

In [ ]:
# Ar in the box - plotly

import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter3d(
    x=X, y=Y, z=Z, mode='markers',
    marker=dict( size=5, color=E, colorscale='Jet', opacity=0.5)
)])

fig.update_layout(
    title='Ar in the Box', width=700, height=700,
    scene=dict(
        xaxis=dict(title='x (nm)', range=[0, 6]),
        yaxis=dict(title='y (nm)', range=[0, 6]),
        zaxis=dict(title='z (nm)', range=[0, 6]),
        aspectratio=dict(x=1, y=1, z=1)
    )
)

fig.show()

# save the figure

#fig.write_html('Ar_in_the_Box.html')

In [ ]:
# velecity space

import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter3d(
    x=U, y=V, z=W, mode='markers',
    marker=dict(size=5, color=E, colorscale='Jet',  opacity=0.5)
)])

fig.update_layout(
    title='velocity space', width=700, height=700,
    scene=dict(
        xaxis=dict(title='vx (m/s)', range=[-1000, 1000]),
        yaxis=dict(title='vy (m/s)', range=[-1000, 1000]),
        zaxis=dict(title='vz (m/s)', range=[-1000, 1000]),
        aspectratio=dict(x=1, y=1, z=1)
    )
)

fig.show()

